In [77]:
import spacy
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import re
import jovian
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from torchtext import data
from torchtext import datasets
from collections import Counter




In [64]:
tweets = pd.read_csv(r'Desktop/tweets_gender_new.csv', encoding = "ISO-8859-1")

In [65]:
print(tweets)
tweets.head()


       ï»¿_unit_id  _golden _unit_state  _trusted_judgments _last_judgment_at  \
0        815719226    False   finalized                   3    10/26/15 23:24   
1        815719227    False   finalized                   3    10/26/15 23:30   
2        815719228    False   finalized                   3    10/26/15 23:33   
3        815719229    False   finalized                   3    10/26/15 23:10   
4        815719230    False   finalized                   3     10/27/15 1:15   
...            ...      ...         ...                 ...               ...   
12889    815757572     True      golden                 259               NaN   
12890    815757681     True      golden                 248               NaN   
12891    815757830     True      golden                 264               NaN   
12892    815757921     True      golden                 250               NaN   
12893    815757985     True      golden                 249               NaN   

       gender  gender:confi

,ï»¿_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.590000e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,Ã¢â¬ÅIt felt like they were my friends and I...,NaN,7471,10/26/15 12:40,6.590000e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.590000e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.590000e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.590000e+17,NaN,NaN


In [66]:
tweets['Profile'] = tweets['description'] + ' ' + tweets['text']

In [81]:
tweets['gender'] = tweets['gender'].replace(['male'], 1)
tweets['gender'] = tweets['gender'].replace(['female'], -1)


tweets.head()

,ï»¿_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone,Profile,encoded
0,815719226,False,finalized,3,10/26/15 23:24,1,1.0000,yes,1.0,12/5/13 1:48,...,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.590000e+17,main; @Kan1shk3,Chennai,i sing my own rhythm. Robbie E Responds To Cri...,"[[1, 2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1,..."
1,815719227,False,finalized,3,10/26/15 23:30,1,1.0000,yes,1.0,10/1/12 13:51,...,C0DEED,Ã¢â¬ÅIt felt like they were my friends and I...,NaN,7471,10/26/15 12:40,6.590000e+17,NaN,Eastern Time (US & Canada),I'm the author of novels filled with family dr...,"[[12, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, ..."
2,815719228,False,finalized,3,10/26/15 23:33,1,0.6625,yes,1.0,11/28/14 11:30,...,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.590000e+17,clcncl,Belgrade,louis whining and squealing and all i absolute...,"[[25, 33, 34, 35, 36, 37, 11, 38, 17, 39, 40, ..."
3,815719229,False,finalized,3,10/26/15 23:10,1,1.0000,yes,1.0,6/11/09 22:39,...,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.590000e+17,"Palo Alto, CA",Pacific Time (US & Canada),"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...","[[45, 12, 1, 14, 46, 47, 11, 48, 14, 49, 50, 5..."
4,815719230,False,finalized,3,10/27/15 1:15,-1,1.0000,yes,1.0,4/16/14 13:23,...,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.590000e+17,NaN,NaN,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,"[[63, 64, 58, 65, 12, 66, 67, 29, 11, 1, 14, 6..."


In [118]:
nlp = spacy.load('en_core_web_sm')

#tokenizing function
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in nlp.tokenizer(nopunct)]


In [119]:
#count number of occurences of each word
counts = Counter()
for index, row in tweets.iterrows():
    counts.update(tokenize(row['text']))


In [120]:
#delete words the occur rarely
for word in list(counts):
    if counts[word] < 2:
        del counts[word]

In [121]:
#we create a vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [122]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [123]:
tweets['encoded'] = tweets['text'].apply(lambda x: np.array(encode_sentence(x, vocab2index)))
tweets.head()

,ï»¿_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone,Profile,encoded
0,815719226,False,finalized,3,10/26/15 23:24,1,1.0000,yes,1.0,12/5/13 1:48,...,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.590000e+17,main; @Kan1shk3,Chennai,i sing my own rhythm. Robbie E Responds To Cri...,"[[1, 2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1,..."
1,815719227,False,finalized,3,10/26/15 23:30,1,1.0000,yes,1.0,10/1/12 13:51,...,C0DEED,Ã¢â¬ÅIt felt like they were my friends and I...,NaN,7471,10/26/15 12:40,6.590000e+17,NaN,Eastern Time (US & Canada),I'm the author of novels filled with family dr...,"[[12, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, ..."
2,815719228,False,finalized,3,10/26/15 23:33,1,0.6625,yes,1.0,11/28/14 11:30,...,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.590000e+17,clcncl,Belgrade,louis whining and squealing and all i absolute...,"[[25, 33, 34, 35, 36, 37, 11, 38, 17, 39, 40, ..."
3,815719229,False,finalized,3,10/26/15 23:10,1,1.0000,yes,1.0,6/11/09 22:39,...,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.590000e+17,"Palo Alto, CA",Pacific Time (US & Canada),"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...","[[45, 12, 1, 14, 46, 47, 11, 48, 14, 49, 50, 5..."
4,815719230,False,finalized,3,10/27/15 1:15,-1,1.0000,yes,1.0,4/16/14 13:23,...,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.590000e+17,NaN,NaN,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,"[[63, 64, 58, 65, 12, 66, 67, 29, 11, 1, 14, 6..."


In [124]:
Counter(tweets['gender'])

Counter({1: 6194, -1: 6700})

In [125]:
from sklearn.model_selection import train_test_split

In [142]:
X = list(tweets['encoded'])
y = list(tweets['gender'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)


In [91]:
class ProfileDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [143]:
train_ds = ProfileDataset(X_train, y_train)
valid_ds = ProfileDataset(X_valid, y_valid)


In [136]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

In [137]:
def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [129]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
val_dl = DataLoader(valid_ds, batch_size = batch_size)

In [130]:
def load_glove_vectors(glove_file=r"Downloads/glove/glove.twitter.27B.100d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [113]:
def get_emb_matrix(pretrained, word_counts, emb_size = 50):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [131]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [132]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [133]:
model = LSTM_glove_vecs(vocab_size, 100, 100, pretrained_weights)

In [134]:
train_model(model, epochs=30, lr=0.1)


AttributeError: 'ProfileDataset' object has no attribute 'y'